# KNeighborsRegressor Demo
This demo illustrates how SIMBSIG can be used for Nearest Neighbors queries, and how the use compares to scikit-learn. 

In [24]:
!pip install simbsig

In [25]:
from simbsig.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor as KNeighborsRegressor_sk
import h5py as h5py
import numpy as np
import os

## Set Parameters

In [26]:
np.random.seed(98)
n_train_samples = 10
n_query_samples = 5
ylow = 0
yhigh = 10
n_dim = 2

## Create Toy Data
### numpy arrays

In [27]:
# numpy arrays
X_train = np.random.uniform(low=-5, high=5, size=(n_train_samples, n_dim))
X_query = np.random.uniform(low=-5, high=5, size=(n_query_samples, n_dim))
y_train = np.random.randint(low=ylow, high=yhigh, size=(n_train_samples))

### hdf5 files
#### Google Colab
If you work from google colab, you can execute the statement below: Of course you can also store the data in a directory more suitable to you.

In [28]:
# when working on colab, google drive can be used to save and read data
from google.colab import drive
drive.mount('/content/drive')

# depending on the structure of your google drive, you might want to choose a different dataset path
dataset_path = '/content/drive/MyDrive/'

ModuleNotFoundError: No module named 'google'

#### Your Computer
Alternatively, if you work from your computer, you can execute the statement below: Of course you can also store the data in a directory more suitable to you.

In [29]:
# when working from your computer, your disk can be used to save and read data
import os
dataset_path = os.path.dirname(os.path.realpath("__file__"))

In [30]:
# hdf5 files using h5py
train_file = f'train.hdf5'
query_file = f'query.hdf5'

with h5py.File(os.path.join(dataset_path, f"{train_file}"), 'w') as f:
    f.create_dataset("X", data=X_train)
    f.create_dataset("y",data=y_train)

with h5py.File(os.path.join(dataset_path, f"{query_file}"), 'w') as f:
    f.create_dataset("X", data=X_query)

## Scikit-Learn


In [31]:
nn_sk = KNeighborsRegressor_sk(n_neighbors=2)

nn_sk.fit(X_train, y_train)
nn_sk.predict(X_query)

array([5., 8., 8., 5., 8.])

## SIMBSIG
### Using numpy arrays and CPU only
SIMBSIG can be used very similar to scikit-learn. In an existing workflow using scikit-learn, which may be on the verge of exceeding runtime or memory requirements, this allows a seamless transition to SIMBSIG.


In [32]:
nn = KNeighborsRegressor(n_neighbors=2)

nn.fit(X_train, y_train)
nn.predict(X_query)

X_train for X_query progress:   0%|          | 0/1 [00:00<?, ?it/s]


array([5., 8., 8., 5., 8.])

### Using hdf5 files and CPU only
If saving the entire data at once in the computer memory using numpy arrays is not reasonable anymore, the hdf5 file format can help. SIMBSIG can use data in hdf5 files, by setting the `mode` argument to `cpu`

In [36]:
nn_hdf5 = KNeighborsRegressor(n_neighbors=2, mode='hdf5')

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

nn_hdf5.fit(train_data)
pred = nn_hdf5.predict(query_data)

# close hdf5 file
train_data.close()
query_data.close()

pred

X_train for X_query progress:   0%|          | 0/1 [00:00<?, ?it/s]


array([5., 8., 8., 5., 8.])

### Using GPU acceleration
If data gets big, the execution time becomes an issue. SIMBSIG features GPU acceleration, by setting the `device` argument to `gpu`. This works with both inputs, numpy arrays and hdf5 files.

In [37]:
nn_hdf5 = KNeighborsRegressor(n_neighbors=2, mode='hdf5', device='gpu')

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

nn_hdf5.fit(train_data)
pred = nn_hdf5.predict(query_data)

# close hdf5 file
train_data.close()
query_data.close()

pred

AssertionError: Torch not compiled with CUDA enabled